### `ENVIRONMENT SETUP`

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 1000)

import warnings
warnings.filterwarnings("ignore")

### `READING FILES`

In [2]:
# Setting working dir
import os
path = 'G:\\ML - Datahack\\HDFC Bank\\Files'

os.chdir(path)

%pwd

'G:\\ML - Datahack\\HDFC Bank\\Files'

In [3]:
train = pd.read_csv('train_d.csv', low_memory=False)
train = train.drop_duplicates()

In [4]:
test = pd.read_csv('test_d.csv', low_memory=False)

In [5]:
dt = pd.read_csv('data_type.csv', low_memory=False)

In [6]:
train.shape

(16400, 446)

In [7]:
test.shape

(20442, 446)

### `DATA TYPES`

In [8]:
dt.head()

,type,feature
0,category,Col29
1,category,Col30
2,category,Col31
3,category,Col34
4,category,Col36


In [9]:
dt.type.unique()

array(['category', 'float', 'int'], dtype=object)

In [10]:
# Categorical features
cat_cols = dt[dt.type == 'category'].feature.values

# Integer features
int_cols = dt[dt.type == 'int'].feature.values

# Numeric features
float_cols = dt[dt.type == 'float'].feature.values

# Train features
cols = train.drop('Col2', axis=1).columns.values

### `RISK RATE FOR CATEGORICAL VARIABLES`

In [11]:
# Find features where unique() in train and test are same
from collections import Counter

rate_cat_col = []

for x in cols:
    if x in cat_cols:
        a = list(np.sort(train[x].unique()))
        b = list(np.sort(test[x].unique()))

        if Counter(a) == Counter(b):
            rate_cat_col.append(x)

print(f'Unique values same for below featutes in Train & Test: \n\n{rate_cat_col}')

Unique values same for below featutes in Train & Test: 

['Col34', 'Col36', 'Col39', 'Col44', 'Col49', 'Col50', 'Col51', 'Col52', 'Col54', 'Col56', 'Col58', 'Col60', 'Col61', 'Col62', 'Col63', 'Col64', 'Col65', 'Col66', 'Col67', 'Col69', 'Col71', 'Col72', 'Col74', 'Col75', 'Col76', 'Col78', 'Col80', 'Col82', 'Col84', 'Col85', 'Col86', 'Col87', 'Col88', 'Col89', 'Col90', 'Col91', 'Col108', 'Col109', 'Col110', 'Col111', 'Col112', 'Col113', 'Col114', 'Col115', 'Col116', 'Col117', 'Col118', 'Col119', 'Col120', 'Col121', 'Col123', 'Col125', 'Col127', 'Col134', 'Col163', 'Col167', 'Col172', 'Col174', 'Col330', 'Col892', 'Col893', 'Col910', 'Col931', 'Col933', 'Col934', 'Col935', 'Col936', 'Col937', 'Col945', 'Col947', 'Col948', 'Col950', 'Col951', 'Col959', 'Col967', 'Col970', 'Col975', 'Col976', 'Col983', 'Col986', 'Col987', 'Col990', 'Col991', 'Col993', 'Col1001', 'Col1002', 'Col1007', 'Col1009', 'Col1010', 'Col1014', 'Col1015', 'Col1025', 'Col1029', 'Col1049', 'Col1050', 'Col1051', 'Col10

In [12]:
# Calculating risk rate for each category and replace in feature
for r_col in rate_cat_col:
    # Calculate risk rate 
    df_rate = train.groupby([r_col, 'Col2']).size()

    x1 = df_rate[:,1]
    x2 = df_rate[:,1] + df_rate[:,0]

    x3 = round(x1/x2*100, 2)

    # Create a df with categories and corr values 
    rate = pd.DataFrame(data = {'category': x3.index.values , 'value': x3.values})
    rate = rate.fillna(0)

    # replace is dataset
    for x,y in zip(rate.category.values, rate.value.values):
        train.loc[train[r_col] == x, r_col] = y
        test.loc[test[r_col] == x, r_col] = y

### `RISK RATE FOR OTHER FEATURES IF ANY`

In [13]:
# Any other feature in dataset having unique values same in test and train
rate_other_col = []

for x in cols:
    if x not in rate_cat_col:
        a = list(np.sort(train[x].unique()))
        b = list(np.sort(test[x].unique()))

        if Counter(a) == Counter(b):
            rate_other_col.append(x)

print(f'Unique values same for below featutes in Train & Test: \n\n{rate_other_col}')

Unique values same for below featutes in Train & Test: 

[]


### `IMPUTING MISSING VALUES`

In [14]:
# Checking missing in int64 features
for x in int_cols:
    if x in cols:
        a = train[x].isnull().sum()
        b = test[x].isnull().sum()
        
        if (a!=0) | (b!=0):
            print(f'Feature {x} | NaN in Train: {a} | NaN in Test: {b}')

In [15]:
# Checking missing in categorical features
for x in cat_cols:
    if x in cols:
        a = train[x].isnull().sum()
        b = test[x].isnull().sum()
        
        if (a!=0) | (b!=0):
            print(f'Feature {x} | NaN in Train: {a} | NaN in Test: {b}')

In [16]:
# Checking missing in float features (only features with > 20%)
float_miss = []

for x in float_cols:
    if x in cols:
        a = round(train[x].isnull().sum()/len(train)*100,1)
        b = round(test[x].isnull().sum()/len(test)*100,1)
        
        if (a > 20):
            float_miss.append(x)
            print(f'Feature {x} | NaN in Train: {a}% | NaN in Test: {b}%')

In [17]:
if len(float_miss) > 0:
    train.drop(columns=float_miss, axis=1, inplace=True)
    test.drop(columns=float_miss, axis=1, inplace=True)

In [18]:
#Impute the values using scikit-learn SimpleImpute Class
from sklearn.impute import SimpleImputer

df = train.drop('Col2', axis=1)
cols = df.columns.values

imp_mean = SimpleImputer( strategy='median') #for median imputation replace 'mean' with 'median'

imp_mean.fit(df)

imputed_train_df = imp_mean.transform(df)

train[cols] = imputed_train_df

In [19]:
dft = test.drop('Col1', axis= 1)

imp_mean.fit(dft)

imputed_test_df = imp_mean.transform(dft)

test[cols] = imputed_test_df

### `OUTLIER DETECTION`

In [20]:
# from sklearn.ensemble import IsolationForest

# df = train

# X = train.drop('Col2', axis=1) 
# y = train.Col2

# model = IsolationForest(random_state=0)

# model.fit(X,y)

In [21]:
# df['flag'] = model.predict(df.drop('Col2', axis=1))

In [22]:
# df = df[df.flag==1]
# train = df.drop('flag', axis=1)

In [23]:
# train.shape

In [24]:
# test.shape

### `SAVING FILE TO DISK`

In [25]:
import os
path = 'G:\\ML - Datahack\\HDFC Bank\\Files'

os.chdir(path)

%pwd

'G:\\ML - Datahack\\HDFC Bank\\Files'

In [26]:
train.to_csv('done_train.csv', index=False)
test.to_csv('done_test.csv', index=False)